Strict Schema and data validation
Type coercion

In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="gemma2-9b-it")

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

In [3]:
class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age : int = Field(description="Age of the person")
    city: str = Field(description="Name of City the person belongs to")
    description: str = Field(description="Description of the person")

In [4]:
parser=PydanticOutputParser(pydantic_object=Person)

temp=PromptTemplate(
    template="Give the information about the person : {character}\n {format_instructions}",
    input_variables=["character"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

In [5]:
parser

PydanticOutputParser(pydantic_object=<class '__main__.Person'>)

In [6]:
temp

PromptTemplate(input_variables=['character'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "title": "Age", "type": "integer"}, "city": {"description": "Name of City the person belongs to", "title": "City", "type": "string"}, "description": {"description": "Description of the person", "title": "Description", "type": "string"}}, "required": ["name", "age", "city", "description"]}\n```'

In [7]:
chain = temp | llm | parser

In [8]:
chain.invoke({"character":"Bruce Wayne"})

Person(name='Bruce Wayne', age=39, city='Gotham City', description='Billionaire industrialist and philanthropist by day, vigilante crime fighter known as Batman by night.')

In [10]:
chain.invoke({"character":"Peter Parker"})

Person(name='Peter Parker', age=17, city='New York City', description='A teenager balancing high school with his secret identity as Spider-Man, a superhero who gained spider-like powers after being bitten by a radioactive spider.')